In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
all_matches = []

In [3]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [4]:
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]

links = [l.get("href") for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]

In [5]:
from io import StringIO
import time
from functools import reduce

In [6]:

for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
    team_data = requests.get(team_url)
    try:
      matches = pd.read_html(StringIO(str(team_data.text)), match="Scores & Fixtures")[0]
      matches = matches[matches["Comp"] == "Premier League"]
    except ValueError:
        continue
    soup = BeautifulSoup(team_data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and 'all_comps/shooting/' in l]
    sh_link = links[0].split('/')[:-3] + ['c9', 'shooting', f'{team_name}-Match-Logs-Premier-League']
    shooting_data = requests.get(f"https://fbref.com{'/'.join(sh_link)}")
    pas_link = links[0].split('/')[:-3] + ['c9', 'passing', f'{team_name}-Match-Logs-Premier-League']
    passing_data = requests.get(f"https://fbref.com{'/'.join(pas_link)}")
    def_link = links[0].split('/')[:-3] + ['c9', 'defense', f'{team_name}-Match-Logs-Premier-League']
    defense_data = requests.get(f"https://fbref.com{'/'.join(def_link)}")
    poss_link = links[0].split('/')[:-3] + ['c9', 'possession', f'{team_name}-Match-Logs-Premier-League']
    possession_data = requests.get(f"https://fbref.com{'/'.join(poss_link)}")
    try:
        shooting = pd.read_html(StringIO(str(shooting_data.text)), match="Shooting")[0]
    except ValueError:
        continue
    try:
        passing = pd.read_html(StringIO(str(passing_data.text)), match="Passing")[0]
    except ValueError:
        continue
    try:
        defense = pd.read_html(StringIO(str(defense_data.text)), match="Defensive Actions")[0]
    except ValueError:
        continue
    try:
        possession = pd.read_html(StringIO(str(possession_data.text)), match="Possession")[0]
    except ValueError:
        continue
    shooting.columns = shooting.columns.droplevel()
    passing.columns = passing.columns.droplevel()
    defense.columns = defense.columns.droplevel()
    possession.columns = possession.columns.droplevel()
    try:
        shooting_data = shooting[["Date", "Sh", "SoT", "Dist", "SoT%", "G/Sh", "G/SoT", "npxG", "npxG/Sh", "G-xG", "np:G-xG"]]
        passing_data = passing[["Date", "Cmp", "Att", "Cmp%", "TotDist", "PrgDist", "Ast", "xAG", "xA", "KP"]]
        defense_data = defense[["Date", "Tkl", "TklW","Def 3rd","Mid 3rd", "Att 3rd", "Blocks", "Sh", "Pass", "Int", "Tkl+Int", "Clr", "Err"]]
        possession_data = possession[["Date", "Touches", "Def Pen", "Def 3rd", "Mid 3rd", "Att 3rd", "Att Pen", "Live"]]
        data_frames = [matches, shooting_data, passing_data, defense_data, possession_data]
        
        teams_data = reduce(lambda  left,right: pd.merge(left,right,on='Date'), data_frames)
        # matches1 = matches.merge(shooting, on="Date")
        # matches2 = matches1.merge(passing, on="Date")
        # matches3 = matches2.merge(defense, on="Date")
        # teams_data = matches3.merge(possession, on="Date")
    except ValueError:
        continue

    teams_data["Season"] = "2023-2024"
    teams_data["Team"] = team_name
    all_matches.append(teams_data)
    time.sleep(1)

In [8]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df


date   time            comp         round  day venue result gf ga  \
0   2023-08-11  20:00  Premier League   Matchweek 1  Fri  Away      W  3  0   
1   2023-08-19  20:00  Premier League   Matchweek 2  Sat  Home      W  1  0   
2   2023-08-27  14:00  Premier League   Matchweek 3  Sun  Away      W  2  1   
3   2023-09-02  15:00  Premier League   Matchweek 4  Sat  Home      W  5  1   
4   2023-09-16  15:00  Premier League   Matchweek 5  Sat  Away      W  3  1   
..         ...    ...             ...           ...  ...   ...    ... .. ..   
33  2024-04-24  20:00  Premier League  Matchweek 29  Wed  Away      L  2  4   
34  2024-04-27  15:00  Premier League  Matchweek 35  Sat  Away      L  1  5   
35  2024-05-04  15:00  Premier League  Matchweek 36  Sat  Home      L  1  3   
36  2024-05-11  15:00  Premier League  Matchweek 37  Sat  Away      L  0  1   
37  2024-05-19  16:00  Premier League  Matchweek 38  Sun  Home      L  0  3   

           opponent  ...  err  touches  def pen  def 3rd_y mid 3rd_y  \
0           Burnley  ...    2      835       75        267       412   
1     Newcastle Utd  ...    1      788       65        212       411   
2     Sheffield Utd  ...    0      893       40        153       440   
3            Fulham  ...    0      838       64        222       483   
4          West Ham  ...    0      837       49        148       382   
..              ...  ...  ...      ...      ...        ...       ...   
33   Manchester Utd  ...    0      436       78        196       170   
34    Newcastle Utd  ...    0      533       85        188       237   
35  Nott'ham Forest  ...    0      637       48        168       298   
36          Everton  ...    0      524       84        245       186   
37        Tottenham  ...    0      350       58        161       129   

   att 3rd_y att pen live     season              team  
0        158      23  835  2023-2024   Manchester City  
1        174      21  788  2023-2024   Manchester City  
2        304      53  892  2023-2024   Manchester City  
3        138      20  837  2023-2024   Manchester City  
4        311      49  837  2023-2024   Manchester City  
..       ...     ...  ...        ...               ...  
33        73       9  436  2023-2024  Sheffield United  
34       116      19  533  2023-2024  Sheffield United  
35       179      18  636  2023-2024  Sheffield United  
36        99      20  524  2023-2024  Sheffield United  
37        62       8  350  2023-2024  Sheffield United  

[760 rows x 69 columns]

In [9]:
match_df.to_csv("matches23-24.csv")

array(['Manchester City', 'Arsenal', 'Liverpool', 'Aston Villa',
       'Tottenham Hotspur', 'Chelsea', 'Newcastle United',
       'Manchester United', 'West Ham United', 'Crystal Palace',
       'Brighton and Hove Albion', 'Bournemouth', 'Fulham',
       'Wolverhampton Wanderers', 'Everton', 'Brentford',
       'Nottingham Forest', 'Luton Town'], dtype=object)

Index(['date', 'time', 'comp', 'round', 'day', 'venue', 'result', 'gf', 'ga',
       'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain', 'formation',
       'referee', 'match report', 'notes', 'sh', 'sot', 'dist', 'fk', 'pk',
       'pkatt', 'team'],
      dtype='object')